In [1]:
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/MED-PHARM/MED-PHARM.csv", encoding='cp949')

# 2. "영업중" 약국만 필터링
df_active = df[df["영업상태명"] == "영업/정상"].copy()

# 3. 도로명주소 또는 지번주소에서 '구' 추출
def extract_gu(addr):
    if pd.isna(addr):
        return None
    for word in ["강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구",
                 "노원구","도봉구","동대문구","동작구","마포구","서대문구","서초구",
                 "성동구","성북구","송파구","양천구","영등포구","용산구","은평구",
                 "종로구","중구","중랑구"]:
        if word in addr:
            return word
    return None

df_active["gu"] = df_active["도로명주소"].apply(extract_gu)
df_active.loc[df_active["gu"].isna(), "gu"] = df_active["지번주소"].apply(extract_gu)

# 4. 구별 약국 수 집계
pharm_counts = df_active.groupby("gu").size().reset_index(name="pharmacy_count")
pharm_counts

,gu,pharmacy_count
0,강남구,559
1,강동구,308
2,강북구,169
3,강서구,290
4,관악구,242
5,광진구,181
6,구로구,199
7,금천구,144
8,노원구,239
9,도봉구,126


In [3]:
# 5. 평균 대비 점수화 (평균=50점, 0~100 clip)
avg = pharm_counts["pharmacy_count"].mean()
pharm_counts["pharmacy_score"] = (pharm_counts["pharmacy_count"] / avg) * 50
pharm_counts["pharmacy_score"] = pharm_counts["pharmacy_score"].clip(0, 100)

# 6. 저장
pharm_counts.to_csv("../../result/MED/MED-PHARM_result.csv", index=False, encoding="utf-8-sig")

pharm_counts.sort_values("pharmacy_score", ascending=False).head()

,gu,pharmacy_count,pharmacy_score
0,강남구,559,100.000000
17,송파구,403,87.033518
1,강동구,308,66.516932
14,서초구,294,63.493435
10,동대문구,291,62.845543
